<a href="https://colab.research.google.com/github/luciosjodin/ProcesamientoDelHabla/blob/master/Trabajo_Pr%C3%A1ctico_1_PH_Sj%C3%B6din_Lucio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajo Práctico 1📚
___
###Sjödin Lucio
___
##Procesamiento del Habla🖥️
___

##Objetivo del trabajo 🎯:
Descargar de una url un archivo pdf, utilizar la librería PyMuPdf y generar una lista en python donde cada elemento es un diccionario con la siguiente información:
{nro de página,
texto}.
___

Para distribuir mejor el desarrollo del código, en el primer bloque instalo la librería PyMUPDF y en el segundo bloque están las importaciones a las librerías requeridas para descargar y manejar el archivo pdf.

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 75.5 MB/s eta 0:00:00


In [ ]:
import fitz  # Esto es la importación de PyMuPDF
import requests  # Request es para descargar el pdf desde una url
import re  # Librería para detectar los encabezados, pies de página, números romanos, etc

Por una cuestión de prolijidad en el código, divido cada bloque en diferentes funciones que luego serán llamadas desde el main.

Como primer bloque de función tenemos la de descarga del archivo pdf.

In [ ]:
def descargar_pdf(url, nombre_archivo):
    try:
      response = requests.get(url, stream=True) # El stream=true no hace mucha falta en este caso particular porque el archivo es pequeño, pero en archivos grandes lo divide y descarga por partes pequeñas
      response.raise_for_status()

      with open(nombre_archivo, "wb") as file:
        file.write(response.content)
        print(f"PDF descargado satisfactoriamente y almacenado como {nombre_archivo} 👍\n")
    except requests.exceptions.RequestException as e:
      print(f"👎 Error en la descarga del archivo PDF: {e}")

Como segundo bloque de función, verificamos utilizando la librería re, la existencia de encabezados, pies de página, números sueltos, números romanos y URLS.

In [ ]:
def es_titulo_pie_numero_url(linea):
    linea = linea.strip() # El strip elimina espacios al inicio y al final de cada línea. Limpieza
    if re.match(r'^Página \d+$', linea):  # "Página ..."
        return True
    if re.match(r'^[IVXLCDM]+$', linea):  # Números romanos
        return True
    if linea == "El Gaucho Martín Fierro":  # Título principal
        return True
    if linea == "MARTIN FIERRO": #Por página 49, está ese título
      return True
    if re.match(r'^\d+$', linea):  # Números sueltos (había algunos números invisibles en el pdf pero aparecían en la ejecución. Por ejemplo un "4" en página 3)
        return True
    if linea.startswith('http://') or linea.startswith('https://'):  # URLs
        return True
    return False

El tercer bloque de función extrae literalmente las estrofas (párrafos) del texto en formato bloque, para mantener la separación visual del archivo. Porqué? Resulta que originalmente (lo dejo al final de colab) la función extraía el texto plano (sin utilizar "blocks" de pymupdf) y tenía seteado separar párrafos cada 6 líneas, algo super útil al asumir que todo el archivo eran sextinas. Sin embargo al imprimir los resultados vi que había errores y al revisar el pdf en profundidad encontré que existían también estrofas de 4, 5, 6 y 10 líneas. Por lo que utilizar un tamaño fijo de líneas no era una opción viable. La IA Grok me había mencionado en un momento que una alternativa si queriá manejar las distancias visuales era usar "blocks", por lo que investigué sobre eso y finalmente decidí que era la mejor opción.

Lo bueno, ahora los párrafos están bien separados sin importar la cantidad de líneas (se puede observar uno de 5 en página 9 y varios de 4 en página 27).

Lo malo de este enfoque, los párrafos incompletos (que comienzan en un página y terminan en otra) son directamente cortados y tratados como párrafos independientes.

No termino de quedarme tranquilo con este enfoque de bloques por esos cortes en los incompletos pero al mismo tiempo esas partes corresponden a páginas distintas por lo que tampoco me parece mal que sean elementos separados dentro de la lista de diccionarios.

In [ ]:
def extraer_estrofas_por_bloques(doc, pagina_inicio=3):
    lista_parrafos = []

    print("Extrayendo los párrafos... ⏱️\n")

    for numero_pagina in range(pagina_inicio, doc.page_count):
        pagina = doc.load_page(numero_pagina)
        texto = pagina.get_text("blocks")  # Extrae el texto como bloques

        lineas_estrofa_actual = []
        numero_pagina_actual = numero_pagina + 1

        for bloque in texto: # Bloque es una tupla: (x0, y0, x1, y1, texto (este es el elemento 4 de la tupla), block_no, block_type)
            texto_bloque = bloque[4] # Accedo al texto indicando por índice el elemento 4
            lineas = texto_bloque.split('\n') # Separa las líneas por espacio

            for linea in lineas:
                linea_limpia = linea.strip()
                if not es_titulo_pie_numero_url(linea_limpia) and linea_limpia: # Si no es título, pie, etc y está limpia la línea, se almacena en la estrofa
                    lineas_estrofa_actual.append(linea_limpia)
                elif not linea_limpia and lineas_estrofa_actual: # Si la línea está vacía y ya hay elementos en la estrofa, se corta la misma y se guarda
                    lista_parrafos.append({"nro_pag": numero_pagina_actual, "texto": '\n'.join(lineas_estrofa_actual)})
                    lineas_estrofa_actual = [] # Y se vacía para volver a comenzar el bucle
                elif es_titulo_pie_numero_url(linea_limpia) and lineas_estrofa_actual: # Si es un título, pie, etc y la estrofa tiene líneas, se guarda la estrofa actual
                     lista_parrafos.append({"nro_pag": numero_pagina_actual, "texto": '\n'.join(lineas_estrofa_actual)})
                     lineas_estrofa_actual = [] # También se vacía

    if lineas_estrofa_actual: # En este caso si la lista no está vacía pero tampoco se ha cerrado y quedan líneas al final de la página, las une y cierra el bloque como un párrafo
            lista_parrafos.append({"nro_pag": numero_pagina_actual, "texto": '\n'.join(lineas_estrofa_actual)})

    print("Retornando párrafos! 🛠️\n")

    return lista_parrafos

Este bloque corresponde a la función principal para ejecutar el programa, la función main(). Aquí se escribe el url para la descarga, se asigna un nombre al archivo guardado de manera local, se ejecuta la descarga, se abre el archivo, se extraen los párrafos en la lista de diccionarios, se cierra el archivo y luego se ejecuta un for para mostrar la totalidad de los elementos.

In [ ]:
def main():
    url = "https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf"
    ruta_pdf = "Martin_Fierro.pdf"

    descargar_pdf(url, ruta_pdf)

    doc = fitz.open(ruta_pdf)

    lista_parrafos = extraer_estrofas_por_bloques(doc)

    doc.close()


    print(f"Cantidad total de párrafos extraídos 👉: {len(lista_parrafos)}\n")

    print("La lista completa es 👇:\n")
    for item in lista_parrafos:
        print(item)

    print("\nFin de ejecución del programa 😁")

#Ejecución del programa ☝️🤓

In [ ]:
if __name__ == "__main__":
    main()

PDF descargado satisfactoriamente y almacenado como Martin_Fierro.pdf 👍

Extrayendo los párrafos... ⏱️

Retornando párrafos! 🛠️

Cantidad total de párrafos extraídos 👉: 418

La lista completa es 👇:

{'nro_pag': 4, 'texto': 'Aquí me pongo a cantar\nal compás de la vigüela,\nque el hombre que lo desvela\nuna pena estrordinaria,\ncomo la ave solitaria\ncon el cantar se consuela.'}
{'nro_pag': 4, 'texto': 'Pido a los santos del cielo\nque ayuden mi pensamiento:\nles pido en este momento\nque voy a cantar mi historia\nme refresquen la memoria\ny aclaren mi entendimiento.'}
{'nro_pag': 4, 'texto': 'Vengan santos milagrosos,\nvengan todos en mi ayuda,\nque la lengua se me añuda\ny se me turba la vista;\npido a mi Dios que me asista\nen una ocasión tan ruda.'}
{'nro_pag': 4, 'texto': 'Yo he visto muchos cantores,\ncon famas bien otenidas\ny que después de alquiridas\nno las quieren sustentar:\nparece que sin largar\nse cansaron en partidas.'}
{'nro_pag': 4, 'texto': 'Mas ande otro criollo pasa

Aquí debajo dejo como constancia las dos funciones que fueron reemplazadas al utilizar el criterio "blocks" para extraer las estrofas.
Este código funcionaba muy bien pero cometía el error de asumir que todas las estrofas eran de 6 versos, cosa que en este caso no es así.
Si bien la mayoría del Martin Fierro está escrito en sextinas, hay otro tipo de estrofas poéticas que no respetan esa estructura y debían ser contempladas y bien separadas mediante el programa. Con estas funciones se asiganaba como número de página al número de página de la primer línea del párrafo.

In [ ]:
'''
def extraer_lineas_poema(doc, pagina_inicio=3):
    """Extrae líneas del poema desde la página indicada, con número de página."""
    lines_with_page = []
    for page_num in range(pagina_inicio, doc.page_count):
        page = doc.load_page(page_num)
        text = page.get_text()
        lines = [line.strip() for line in text.split('\n') if line.strip() and not es_titulo_o_pagina(line)]
        for line in lines:
            lines_with_page.append((page_num + 1, line))  # (página, línea)
    return lines_with_page

def agrupar_en_estrofas(lines_with_page, tamano_estrofa=6):
    """Agrupa líneas en estrofas de tamaño fijo, con página de la primera línea."""
    lista_parrafos = []
    i = 0
    while i < len(lines_with_page):
        estrofa = lines_with_page[i:i+tamano_estrofa]
        if len(estrofa) == tamano_estrofa:
            nro_pag = estrofa[0][0]
            texto = '\n'.join([line[1] for line in estrofa])
            lista_parrafos.append({"nro_pag": nro_pag, "texto": texto})
            i += tamano_estrofa
        else:
            print(f"Advertencia: {len(estrofa)} líneas restantes, no una estrofa completa. Ignorando.")
            break
    return lista_parrafos
'''

'\ndef extraer_lineas_poema(doc, pagina_inicio=3):\n    """Extrae líneas del poema desde la página indicada, con número de página."""\n    lines_with_page = []\n    for page_num in range(pagina_inicio, doc.page_count):\n        page = doc.load_page(page_num)\n        text = page.get_text()\n        lines = [line.strip() for line in text.split(\'\n\') if line.strip() and not es_titulo_o_pagina(line)]\n        for line in lines:\n            lines_with_page.append((page_num + 1, line))  # (página, línea)\n    return lines_with_page\n\ndef agrupar_en_estrofas(lines_with_page, tamano_estrofa=6):\n    """Agrupa líneas en estrofas de tamaño fijo, con página de la primera línea."""\n    lista_parrafos = []\n    i = 0\n    while i < len(lines_with_page):\n        estrofa = lines_with_page[i:i+tamano_estrofa]\n        if len(estrofa) == tamano_estrofa:\n            nro_pag = estrofa[0][0]\n            texto = \'\n\'.join([line[1] for line in estrofa])\n            lista_parrafos.append({"nro_pa

#Conclusión
El presente trabajo realmente fue desafiante porque en primera instancia parecía más o menos sencillo, si la estructura del pdf tenía espacios claros para separar los párrafos y eran todos del mismo tamaño. Al descubrir que esto no era así he tenido que investigar bastante y reinterpretar la consigna. Finalmente el programa a mi criterio funciona muy bien ya que en ningún momento los párrafos son desfasados (cosa que pasaba antes), por lo tanto si bien existen párrafos cortos porque son cortados, ninguno genera problemas con el próximo. Obviamente me queda la duda y ganas de aprender si se pudiese tratar de otra manera el documento para separar los párrafos y al mismo tiempo unir los cortados con la parte que sigue en otra página, sin tener que tratar cada caso de forma individual y contemplando la existencia de un tamaño dinámico para los párrafos.

#Para finalizar dejo las referencias a herramientas utilizadas en este Colab:

Archivo pdf: https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf

Documentación de PyMUPDF: https://pymupdf.readthedocs.io/en/latest

Gemini incorporado dentro de Google Colab

IA Grok: https://grok.com

Emojis: https://es.piliapp.com/emoji/list